In [6]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
from rl_algorithms import QPlayer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
n_episodes = 20000
turns = np.array(['X','O'])

q_player = QPlayer(0.05, 0.99, 0.2)
env = TictactoeEnv()

average_reward = 0
rewards = []
for episode in range(n_episodes):
    if episode % 1000 == 0:
        print('Current episode: %d' % episode)
    env.reset()
    grid, _, __ = env.observe()
    other_player = OptimalPlayer(epsilon=0.5, player=turns[1])

    for j in range(9):
        if env.current_player == other_player.player:
            move = other_player.act(grid)
        else:
            move = q_player.act(grid)

        grid, end, winner = env.step(move, print_grid=False)
        if env.current_player == other_player.player and not end:
            q_player.learn(grid, 0, False)

        if end:
            reward = env.reward(turns[0])
            average_reward += reward
            q_player.learn(grid, reward, True)

            if episode % 50 == 49:
                average_reward /= 50
                rewards.append(average_reward)
                average_reward = 0

            turns = turns[::-1]
            break

Current episode: 0
Current episode: 1000
Current episode: 2000
Current episode: 3000
Current episode: 4000
Current episode: 5000
Current episode: 6000
Current episode: 7000
Current episode: 8000
